### 20184645 김영찬
### 웹크롤링 후 데이터베이스 저장

In [1]:
# 크롤링에 필요한 각 라이브러리, 모둘 임포트 및 설치
#!pip install selenium
#!pip install bs4
#!pip install flask
import re # 정규표현식을 위한 임포트
import time # 웹 페이지가 뜨는 동안 시간지연을 하기 위해 임포트
import urllib
import sqlite3
from selenium import webdriver
from bs4 import BeautifulSoup
from flask import Flask,render_template

In [2]:
# 먼저, 크롬을 켜 알바몬 사이트에 접속
browser = webdriver.Chrome("./chromedriver.exe")
browser.get("https://www.albamon.com/main")

In [3]:
# 맨 처음 홈페이지에 들어가고 기업/개인 을 선택하는 팝업에서 개인을 클릭
main=browser.find_element_by_id("btn_MType")
main.click()

# 크롤링 할 위치인 내가 사는 지역인 경기 - 수원시 장안구 - 장안구 전체 를 선택 후 검색
browser.find_element_by_link_text("경기").click()
browser.find_element_by_link_text("수원시 장안구").click()
time.sleep(1)
browser.find_element_by_id("dev_area_dong_code_B190").click()
s_location=browser.find_element_by_class_name("sch")
s_location.click()
time.sleep(1) # 위치가 바뀌고 브라우저가 새로고침 되는동안 다음 동작이 실행되면 오류가 나기떄문에 1초의 시간지연을 두었음.

In [4]:
# alba 데이터베이스 연결
conn = sqlite3.connect('alba.db')

cursor = conn.cursor()

# alba_data 테이블 생성
cursor.execute('''
              create table alba_data(area text, subject text, money text, time text, recently text);
''') 

OperationalError: table alba_data already exists

In [ ]:
# 본격적으로 필요한 자료들을 불러오기 위해 뷰티풀 숩 객체 생성
# current_url은 내가 직접 페에지를 들어간것이 아니고 위의 코드에서 자동으로 페이지에 들어갔기 떄문에
# 현재 탭의 url을 가져와 준다.

base_url = urllib.request.urlopen(browser.current_url).read()
soup = BeautifulSoup(base_url,"html.parser")

ths = []  # 테이블의 헤드를 담기 위한 리스트
tds = []
alba_data = [] # 알바의 정보들을 담기위함.

# 크롤링해온 정보들의 필요없는문자들(ex.\n, \r, \t)
def clean_text(data):
    data1 = re.sub('\n|\r|\t|  ','',str(data))
    data2 = re.sub('\r\n','',data1)
    return data2


# 지역 선택후 나온 페이지의 하단의 알바 정보들이 모영있는 테이블을 find
wrap = soup.find('div',{"class":"gListWrap"}) 
table = wrap.find('table')
t_head = table.find('thead').find_all('th') # 찾은 테이블의 헤드(카테고리?(지역/금여금액/직업명 등등..))
t_body = table.find('tbody') # 찾은 테이블의 바디(알바들의 직접적인 정보)

# 테이블의 헤드를 하니싹 ths리스트에 담는다.
for th in t_head:
    ths.append(th.get_text())

# 알바 정보들의 직접적인 정보들을 수집
for tr in t_body.find_all('tr'):
    area = tr.find('td',{'class':'area'}).find('div') # 알바를 하는 지역의 정보를 담고 있는 div태그
    area.span.decompose() # 이 div 태그에는 span태그안에 다른 텍스트가 있는데 이 택스트를 제외한 값만 가져오기 위하여 사용
    subject_name = tr.find('td',{'class':'subject'}).find('p',{'class':'cName'}).text # p태그 안의 cName(기업명)을 수집
    subject_ctit = tr.find('td',{'class':'subject'}).find('p',{'class':'cTit'}).text # p태그 안의 cTit(모집 제목)을 수집
    pay_money = tr.find('td',{'class':'pay'}).find('p',{'class':'money'}).find("img")["alt"] 
    #일급 / 시급 / 월급과 같은 급여 지급방식이 나뉘어져 있는데 이 부분은 img 태그안의 alt속성에 표기되어있어 리스트의 ["alt"]의 택스트만 수집
    pay_won = tr.find('td',{'class':'pay'}).find('p',{'class':'won'}).text # p태그 안의 won(급여)을 수집
    # for문을 한개만 사용하여 접근하려 했지만 급여 시간의 td태그에는 class 이름이 없었기 떄문에 결국 for문을 한개 더사용하여
    # 3번째 위치에 있는 급여 시간 수집
    tds = list(tr.find_all('td'))
    for td in tds:
        time = tds[3].text
    # 등록일 정보 수집
    recently = tr.find('td',{'class':'recently'}).text
    # 위에서 수집한 알바정보들을 불필요한 문자들을 제거한 후 alba_data 리스트에 담는다.
    alba_data.append([clean_text(area.text),clean_text(subject_name)+clean_text(subject_ctit),clean_text(pay_money)+clean_text(pay_won),clean_text(time),clean_text(recently)])
    
    cursor.execute('''
    insert into alba_data(area, subject, money, time, recently) values (?,?,?,?,?)''',
                  (clean_text(area.text), clean_text(subject_name)+clean_text(subject_ctit), clean_text(pay_money)+clean_text(pay_won),
                  clean_text(time),clean_text(recently)))

conn.commit()

In [5]:
# 데이터베이스에서 area 속성이 ~조원동으로 끝나는 행만 검색
cursor.execute('select * from alba_data where area like "%조원동"')
rows = cursor.fetchall()

for row in rows:    
    print("area : %s, subject: %s, money : %s, time : %s, recently : %s" % 
          (row[0],row[1],row[2],row[3],row[4]))
    print("\n")

conn.close()

area : 장안구 조원동, subject: 쿠팡풀필먼트서비스 [여주쿠팡]#품질관리&검수센터#쉬운업무#월최대270만, money : 월급2,705,195원, time : 19:00~04:00, recently : 34분전


area : 장안구 조원동, subject: [쿠팡/선착순]박스분류(단순) #하루14만6천원(식사제공), money : 일급146,375원, time : 21:00~06:00, recently : 34분전


area : 장안구 조원동, subject: #초보가능 #동반근무 #하루알바 주급65만가능/원하는요일/셔틀제공/단기/쿠팡/용인, money : 일급96,750원, time : 시간협의, recently : 34분전


area : 장안구 조원동, subject: #CFS #쾌적한환경 #무료셔틀 #초보환영 #동 [쿠팡]안성5,6,7 #입사축하금100만원 #HUB #선착순, money : 월급2,845,000원, time : 09:00~18:00, recently : 35분전


